<a href="https://colab.research.google.com/github/hungpham13/Laptop-EDA/blob/main/Data_Cleaning_and_Integrating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/hungpham13/Laptop-EDA.git

Cloning into 'Laptop-EDA'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 253 (delta 151), reused 128 (delta 58), pack-reused 0
Receiving objects: 100% (253/253), 25.39 MiB | 9.86 MiB/s, done.
Resolving deltas: 100% (151/151), done.
Checking out files: 100% (28/28), done.


# Data cleaning - Data preprocessing

In [11]:
def text_facet(df, col, print_all=True,ascending=False):
    if print_all:
        with pd.option_context('display.max_rows', 1000, 'display.max_columns', None):
            print(f"---------- {col} ------------")
            print(df[col].value_counts(dropna=False,ascending=ascending))
    else:
        print(f"---------- {col} ------------")
        print(df[col].value_counts(dropna=False,ascending=ascending))
        print(f"Number of null: {df[col].isna().sum()}")

def get_element_fptshop(id):
    r1 = cleaned_fpt.iloc[id]
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(r1)

def lower_all_string(df):
    '''lower all values in string columns of dataframe df'''
    for c in df.columns:
        if df[c].dtypes == np.object:
            df[c] = df[c].str.lower()

data_path = 'C:/Users/admin/Downloads/Laptop-EDA/scraped_data/'
all_brands = ['apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft',
              'huawei','avita','gigabyte','fujitsu']

In [12]:
retained_cols = ['Comment_count','Price', 'Rating', 'Rating_count']
mapping_cols = ['Series','Brand','cpu_brand','cpu_code','ram_size',\
                'storage_size','displ_size','displ_rate']
other_cols = ['Battery', 'GPU', 'OS', 'P/N',  'displ_resolution', 'hdmi',\
            'Release_year', 'Surface', 'Weight', 'backlit_keyboard', \
            'lan', 'length', 'ram_upgradable', 'screen_technology',\
            'sd_card', 'storage_extra_slot', 'storage_type', \
            'headphone_jack', 'thickness', 'type_c', 'usb_a', 'width']
all_cols = retained_cols + mapping_cols + other_cols

## Thegioididong - Dienmayxanh
result: no screen technology

In [23]:
import re
import numpy as np

def clean_thegioididong(source):
    source.dropna(subset=['ports'],inplace=True)
    result = pd.DataFrame()
    result['P/N'] = source['title'].str.extract(r"\(([^)]+)\)",expand=False).str.strip().str.replace("\.$", "") #replace any dot at the end of P/N
#sử dụng các phương thức chuỗi trong Pandas như str.extract()(trích xuất các chuỗi chứa các chuỗi số([^)]+)\), str.strip()(loại bỏ khoảng trắng), 
# và str.replace()(thay thế bất kỳ dấu chấm (.) nào ở cuối chuỗi trong Series bằng một chuỗi trống) để loại bỏ khoảng trắng
#  không cần thiết và dấu chấm cuối cùng của chuỗi P/N (nếu có), sau đó kết quả được gán vào cột 'P/N' trong DataFrame result.
    result['Rating'] = source['rating'].fillna(0).astype('float64')
    #gán cột 'Rating' trong DataFrame result với các giá trị từ cột 'rating' của DataFrame source, trong đó giá trị null được thay thế bằng 0 
    # và sau đó được chuyển đổi thành kiểu dữ liệu float64.
    result['Rating_count'] = source['review_count'].str.extract(r'(\d+)',expand=False).str.strip().fillna(0).astype('int64')
#gán cột 'Rating_count' trong DataFrame result với các giá trị tương ứng từ chuỗi con đã trích xuất từ cột 'review_count' của DataFrame source
#Các giá trị null được thay thế bằng 0 và sau đó chuyển đổi thành kiểu dữ liệu số nguyên (int64) để biểu diễn số lượng đánh giá.
    result['Comment_count'] = 0
    for i,r in source['cmt_count'].iteritems():
        if pd.isnull(r): # Kiểm tra nếu giá trị của hàng hiện tại trong cột 'cmt_count' là null, thì bỏ qua và chuyển đến hàng tiếp theo.
            continue
        for c in r.split("|"): # Với mỗi phần từ được tách ra từ chuỗi trong cột 'cmt_count' dựa trên dấu "|".

            if re.search(r'([\d.,]+)',c): #Kiểm tra nếu có một chuỗi con trong dãy chứa các số hoặc dấu chấm.
                result['Comment_count'][i] += int(re.search(r'([\d.,]+)',c).group().replace(".",""))
                #Tìm và lấy chuỗi con chứa các số hoặc dấu chấm từ chuỗi hiện tại. Sau đó, loại bỏ dấu chấm nếu có 
                # bằng cách sử dụng .replace(".", "") và chuyển đổi chuỗi số thành kiểu integer. Cuối cùng, giá trị này được thêm vào cột 'Comment_count' tương ứng với hàng 'i' trong DataFrame result.
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['Comment_count'].value_counts(dropna=False))

    

    result['Brand'] = source['title'].str.split().str[1].str.strip().str.lower()
    # Brand = phần tử thứ hai trong title.
#strip(): Loại bỏ các khoảng trắng không cần thiết ở đầu và cuối của từ.

    # print(result['Brand'].value_counts(dropna=False))
    result['Series'] = source['title'].str.extract(r'\w+\s\w+\s([\w\s]+)\s[\w\/d]+',expand=False).str.strip()
    #extract title để ra Series
    #r'\w+\s\w+\s([\w\s]+)\s[\w\/d]+' có ý nghĩa:
#\w+: Tìm một hoặc nhiều ký tự từ bảng chữ cái, số hoặc dấu gạch dưới.
#\s: Định dạng khoảng trắng.
#([\w\s]+): Trích xuất một nhóm các ký tự từ bảng chữ cái, số hoặc khoảng trắng.
#[\w\/d]+: Tìm một chuỗi gồm các ký tự từ bảng chữ cái, số hoặc dấu gạch chéo và d.
    result.loc[(result['Series'].str.split().str[-1].str.len() == 2), 'Series'] =  result['Series'].str.slice(stop=-3)

    #add P/N to Series (except Apple laptop)
    result['Series'] = np.where(result['Brand'] != 'apple', 
                                result[['Series', 'P/N']].fillna('').agg(' '.join, axis=1),
                                result['Series'])
    result.Series[result.Series == 'MacBook Pro'] = 'MacBook Pro M1 2020'
    # for a in zip(result.Series, source.title):
    #     print('{0:40} {1}'.format(a[0],a[1]))

    result['Price'] = source['price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')

    result['cpu_brand'] = source['cpu_name'].str.extract(r'([\w]+)\s',expand=False).str.strip()
    result['cpu_code'] = source['cpu_name'].str.extract(r'-([\w\d\s]+)',expand=False).str.strip().fillna('m1')
    result['cpu_code'] = np.where(result['cpu_code'].str.len() > 10, 
                                    source['cpu_name'].str.extract(r'([\w\d\s]+)-',expand=False).str.split().str.slice(start=1).str.join(' '),
                                    result['cpu_code'])

    result['ram_size'] = source['ram_size'].str.extract(r'(\d+)',expand=False).str.strip().fillna(8).astype('float64')
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['ram_size'].value_counts(dropna=False))

    result['ram_upgradable'] = np.where(source['ram_upgradable'].str.contains("Kh"), "no", "yes")

    text_facet(source,'gpu')
    result['GPU'] = source['gpu'].str.extract(r' - ([\w\d\s-]+),?',expand=False).str.strip()
    text_facet(result,'GPU')

    result['Battery'] = source['battery'].str.extract(r'([\d\s.]+)Wh',expand=False).str.strip().astype('float64')

    result['storage_size'] = np.nan
    result['storage_type'] = np.nan
    result['storage_extra_slot'] = np.nan
    # for i,r in result.loc[result['storage_size'].notnull(),'storage_size'].iteritems():
    for i,r in source['storage'].iteritems():
        if pd.isnull(r):
            continue
        for s in r.split("|"):
            s = s.lower()
            if "hỗ trợ" not in s:
                result['storage_size'][i] = re.search(r'(\d+)\s?[gtb]{2}',s.strip()).group(1)
                if "ssd" in s:
                    result['storage_type'][i] = "SSD"
                elif "hdd" in s:
                    result['storage_type'][i] = "HDD"
            elif "không" not in s: #just have "ho tro", no "khong"
                result['storage_extra_slot'][i] = 'yes'
        if pd.isnull(result['storage_extra_slot'][i]):
                result['storage_extra_slot'][i] = 'no'
    result['storage_size'] = np.where(result['storage_size']==1,1024,result['storage_size'])
    result['storage_size'] = np.where(result['storage_size']==2,2048,result['storage_size'])
    # print(result['storage_size'].value_counts(dropna=False))

    result['displ_size'] = source['displ_size'].str.extract(r'([\d.]+)',expand=False).astype('float64')
    result['displ_resolution'] = source['displ_resolution'].str.extract(r"\(([^)]+)\)",expand=False).str.strip()
    result.displ_resolution = result.displ_resolution.replace('2160x1350','2160 x 1350')
    result.displ_resolution = result.displ_resolution.replace('2240x1400','2240 x 1400')
    result['displ_rate'] = source['displ_rate'].str.extract(r'([\d]+)',expand=False).fillna(60).astype('int64')
    result['screen_technology'] = np.nan
    for i,r in source.displ_tech.iteritems():
        if pd.isnull(r):
            continue
        elif 'ips' in r.lower():
            result['screen_technology'][i] = 'IPS'
        elif 'oled' in r.lower():
            result['screen_technology'][i] = 'OLED'
        elif 'wva' in r.lower():
            result['screen_technology'][i] = 'VA'
        elif 'led' in r.lower():
            result['screen_technology'][i] = 'LED'
        elif 'fhd' in r.lower():
            result['screen_technology'][i] = 'fhd'
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['screen_technology'].value_counts(dropna=False))
    #     print(source.displ_tech[result.screen_technology.isnull()])

    result['Weight'] = source['weight_dims'].str.extract(r"Nặng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['length'] = source['weight_dims'].str.extract(r"Dài ([\d.]+)",expand=False).str.strip().astype('float64')
    result['width'] = source['weight_dims'].str.extract(r"Rộng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['thickness'] = source['weight_dims'].str.extract(r"Dày ([\d.]+)",expand=False).str.strip().astype('float64')

    result['OS'] = source['os'].str.split().str[:2].str.join(" ").str.strip()
    result['Surface'] = np.nan
    for i,r in source['surface'].iteritems():
        if pd.isnull(r):
            continue
        d = {"nhôm":"Aluminum","carbon":"Carbon","kim loại": "Metal", "nhựa":"Plastic","magie":"Magnesium"}
        s = []
        for vn_term in d:
            if vn_term in r.lower():
                s.append(d[vn_term])
        result['Surface'][i] = ', '.join(s)
        
    result['Release_year'] = source['launch']
    result['backlit_keyboard'] = np.where(source['backlight_key'].str.contains('Không'),'no','yes')

    result['hdmi'] = np.where(source['ports'].str.lower().str.contains("hdmi"),'yes','no')
    result['headphone_jack'] = np.where(source['ports'].str.lower().str.contains("jack"),'yes','no')
    result['lan'] = np.where(source['ports'].str.lower().str.contains("lan"),'yes','no')
    result['type_c'] = np.where(source['ports'].str.lower().str.contains("-c"),'yes','no')
    result['usb_a'] = np.where(source['ports'].str.lower().str.contains(r"usb [a32]",regex=True),'yes','no')
    result['sd_card'] = np.where(pd.isnull(source['sd_card']),'no','yes')
    lower_all_string(result)
    return result

In [24]:
import pandas as pd
scraped_tgdd = pd.read_csv(data_path+"tgdd_dmx.csv",index_col='url')
cleaned_tgdd = clean_thegioididong(scraped_tgdd)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(cleaned_tgdd.Series.value_counts(dropna=False))
#     print(cleaned_tgdd)
cleaned_tgdd.to_csv(data_path+'tgdd_dmx_cleaned.csv',encoding='utf-8')

---------- gpu ------------
Card tích hợp|Intel Iris Xe Graphics|Card tích hợp - Intel Iris Xe Graphics                                          65
Card rời|NVIDIA GeForce RTX3050, 4 GB|Card rời - NVIDIA GeForce RTX3050, 4 GB                                        23
Card tích hợp|Intel UHD Graphics|Card tích hợp - Intel UHD Graphics                                                  23
Card rời|Card rời - NVIDIA GeForce RTX3050Ti, 4 GB                                                                   10
Card rời|NVIDIA GeForce RTX 3060, 6 GB|Card rời - NVIDIA GeForce RTX 3060, 6 GB                                       7
Card rời|NVIDIA GeForce MX330, 2 GB|Card rời - NVIDIA GeForce MX330, 2 GB                                             7
Card rời|NVIDIA GeForce GTX 1650 4 GB|Card rời - NVIDIA GeForce GTX 1650 4 GB                                         5
Card rời|NVIDIA GeForce GTX 1650Ti 4 GB|Card rời - NVIDIA GeForce GTX 1650Ti 4 GB                                     5
Card rời|NVI

C:\Users\admin\AppData\Local\Temp\ipykernel_20232\2431104055.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  result['P/N'] = source['title'].str.extract(r"\(([^)]+)\)",expand=False).str.strip().str.replace("\.$", "") #replace any dot at the end of P/N
C:\Users\admin\AppData\Local\Temp\ipykernel_20232\2431104055.py:18: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i,r in source['cmt_count'].iteritems():
C:\Users\admin\AppData\Local\Temp\ipykernel_20232\2431104055.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Comment_count'][i] += int(re.search(r'([\d.,]+)',c).group().replace(".",""))
C:\Users\admin\AppData\Local\Temp\ipykernel_20232\2431104055.py:50: SettingWithCopyWarning: 
A

## Fpt shop
- Rename column to the same format as data form
- Clean `Series`,`displ_rate`
- `Rating`,`Rating_count`: (5,1) to (0,0)

In [28]:
scraped_fpt = pd.read_csv(data_path+"fptshop_scraped.csv")
cleaned_fpt = pd.read_csv(data_path+"fpt-cleaned.csv")
cleaned_fpt.rename(columns={'PN':'P/N',
                            'rating':'Rating',
                            'rating_count':'Rating_count',
                            'comment_count':'Comment_count',
                            'price':'Price',
                            'RAM':'ram_size',
                            'RAM_upgradable':'ram_upgradable',
                            'battery_wh':'Battery',
                            'storage_upgradable':'storage_extra_slot',
                            'screen_size':'displ_size',
                            'display_resolution':'displ_resolution',
                            'display_rating':'displ_rate',
                            'weight':'Weight',
                            'height':'thickness',
                            'surface_material':'Surface',
                            'release_year':'Release_year', 
                            'jack':'headphone_jack',
                            'usb':'usb_a', 
                            'sd':'sd_card',
                            'keyboard_light':'backlit_keyboard'
                            },inplace=True)

lower_all_string(cleaned_fpt)
cleaned_fpt['P/N'] = cleaned_fpt['P/N'].fillna('').astype('object')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_fpt.info()
#     cleaned_fpt.info()
#     print(cleaned_fpt['Series'].value_counts(dropna=False))
for i,row in cleaned_fpt.iterrows():
    r = row.Series
    extract_r1 = re.search(r'([\w\s-]+)[ /]c?o?r?e?\s?[ir\d]{2}[- ]',r)
    extract_r2 = re.search(r'([\w\s-]+)pentium',r)
    extract_r3 = re.search(r'([\w\s-]+)celeron',r)
    extract_r4 = re.search(r'([\w\s-]+)n4020',r)
    extract_r5 = re.search(r'([\w\s-]+)/n4500',r)
    if extract_r1: extract_r = extract_r1.group(1).strip()
    elif extract_r2: extract_r = extract_r2.group(1).strip()
    elif extract_r3: extract_r = extract_r3.group(1).strip()
    elif extract_r4: extract_r = extract_r4.group(1).strip()
    elif extract_r5: extract_r = extract_r5.group(1).strip()
    else: extract_r = r.lower().strip()

    specs_key = [r'\d+gb',r'[\d.]+ghz',r'\d+gpu',r'\d+tb','core i5',r'\d+"']
    for k in specs_key:
        extract_r = re.sub(k, '', extract_r).strip(' -/')
        extract_r = ' '.join(extract_r.split())
    #add P/N
    if row['P/N'] not in extract_r and row.Brand != 'apple':
        extract_r = f"{extract_r} {row['P/N']}"

    # print(f"{r:55}  {row['P/N']:20}  {extract_r}")
    cleaned_fpt.Series[i] = extract_r

#disp_rate
cleaned_fpt.displ_rate = cleaned_fpt.displ_rate.replace("--",60).fillna(60).astype('int64')
print(cleaned_fpt.displ_rate.value_counts(dropna=False))

#rating
cleaned_fpt.Rating[cleaned_fpt.Rating_count == 1] = 0
cleaned_fpt.Rating_count[cleaned_fpt.Rating_count == 1] = 0
cleaned_fpt.Rating_count = cleaned_fpt.Rating_count.fillna(0).astype('int64')

cleaned_fpt.Battery = cleaned_fpt.Battery.str.extract(r'([\d.]+)',expand=False).str.strip().replace("--",np.nan).astype('float64')
cleaned_fpt.Weight = cleaned_fpt.Weight.str.extract(r'([.\d]+)',expand=False).str.strip().replace("--",np.nan).astype('float64')
cleaned_fpt.length = cleaned_fpt.length.str.extract(r'([.\d]+)',expand=False).str.strip().replace("--",np.nan).astype('float64')
cleaned_fpt.width = cleaned_fpt.width.str.extract(r'([.\d]+)',expand=False).str.strip().replace("--",np.nan).astype('float64')
cleaned_fpt.thickness = cleaned_fpt.thickness.str.extract(r'([.\d]+)',expand=False).str.strip().replace("--",np.nan).astype('float64')
# #Battery
# cleaned_fpt.Battery = cleaned_fpt.Battery

#screen_tech
for i,r in cleaned_fpt.screen_technology.iteritems():
    if pd.isnull(r):
        continue
    elif 'ips' in r.lower():
        cleaned_fpt['screen_technology'][i] = 'ips'
    elif 'oled' in r.lower():
        cleaned_fpt['screen_technology'][i] = 'oled'
    elif 'wva' in r.lower():
        cleaned_fpt['screen_technology'][i] = 'va'
    elif 'led' in r.lower():
        cleaned_fpt['screen_technology'][i] = 'led'
    elif 'fhd' in r.lower():
        cleaned_fpt['screen_technology'][i] = 'fhd'
cleaned_fpt.screen_technology = cleaned_fpt.screen_technology.replace("--",np.nan)

cleaned_fpt.displ_resolution = cleaned_fpt.displ_resolution.replace("--",np.nan)
#GPU
cleaned_fpt.GPU = cleaned_fpt.GPU.str.replace(r'\dgb','',regex=True)
#surface
for i,r in cleaned_fpt['Surface'].iteritems():
    if pd.isnull(r):
        continue
    d = {"nhôm":"Aluminum","carbon":"Carbon","kim loại": "Metal", "nhựa":"Plastic", "abs plastic":"Plastic","magie":"Magnesium"}
    s = []
    for vn_term in d:
        if vn_term in r.lower():
            s.append(d[vn_term])
    cleaned_fpt['Surface'][i] = ', '.join(s)
    if not s:
        cleaned_fpt['Surface'][i] = np.nan

cleaned_fpt.Release_year = cleaned_fpt.Release_year.replace('trung quốc',np.nan)
cleaned_fpt.Release_year = cleaned_fpt.Release_year.replace('--',np.nan)

for i,r in cleaned_fpt['OS'].iteritems():
    if pd.isnull(r):
        continue
    if "windows 10" in r:
        cleaned_fpt['OS'][i] = "windows 10"

#cpu_code
cleaned_fpt.cpu_code.fillna('i5',inplace=True)
lower_all_string(cleaned_fpt)
for c in other_cols:
    text_facet(cleaned_fpt,c)

C:\Users\admin\AppData\Local\Temp\ipykernel_20232\3564673205.py:19: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if df[c].dtypes == np.object:
C:\Users\admin\AppData\Local\Temp\ipykernel_20232\3878591056.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_fpt.Series[i] = extract_r
C:\Users\admin\AppData\Local\Temp\ipykernel_20232\3878591056.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

60     100
144     33
120      8
165      3
300      1
90       1
Name: displ_rate, dtype: int64
---------- Battery ------------
65.0     40
NaN      32
45.0     28
180.0     6
150.0     6
240.0     5
42.0      5
200.0     5
90.0      4
48.0      3
120.0     3
37.0      2
59.0      1
130.0     1
100.0     1
170.0     1
30.0      1
39.0      1
33.0      1
Name: Battery, dtype: int64
---------- GPU ------------
intel iris xe graphics             32
intel uhd graphics                 14
amd radeon graphics                13
intel iris plus graphics            5
nvidia geforce rtx3050              5
nvidia geforce mx330                5
intel uhd graphics 600              4
nvidia geforce rtx 3050             4
intel iris graphics                 3
apple m1                            3
nvidia geforce rtx3060              2
nvidia geforce rtx3070              2
apple m1 pro                        2
card onboard                        2
nvidia geforce rtx 3060             2
geforce gtx 1650 

## Cellphone S
- Drop column `'1star', '2star', '3star', '4star', '5star'`
- Rename column `comment_count`
- Change Price to float
- recleaned `Series`, `displ_size`

In [1]:
scraped_cellphones = pd.read_csv(data_path+"cellphones_scraped.csv",index_col='url')
# cleaned_cellphones2 = pd.read_csv("Laptop-EDA/scraped_data/cellphones_cleaned.csv",index_col='url')
cleaned_cellphones = pd.read_csv(data_path+"cellphones-cleaned.csv",index_col='url')

cleaned_cellphones.drop(columns=['1star', '2star', '3star', '4star', '5star'],inplace=True)
cleaned_cellphones.rename(columns={'comment_count':'Comment_count',
},inplace=True)
cleaned_cellphones['Price'] = cleaned_cellphones['Price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_cellphones.info()
#     cleaned_cellphones.info()
#     cleaned_cellphones2.info()
#     # print(scraped_cellphones['Series'].value_counts(dropna=False))
# cleaned_cellphones.head()

for i,cleaned_row in cleaned_cellphones.Series.iteritems():
    r = scraped_cellphones.item_name[i]
    if not isinstance(r, str):
        # print(r)
        r = r[0]
    black_key = [r'\d+gb',r'\d+inch',r'\d+gpu',r'\d+tb',
                 'laptop','chính','hãng','việt','nam','nhập','khẩu','cũ','đẹp','bạc','mqd32',
                 'apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft','huawei','avita','gigabyte']
    cpu_key = ['core','ryzen']
    specs_key = [r'\d+\sgb',r'[\d.]+\sinch',r'\d+\sgpu',r'\d+\stb',
                 'i\d [\d.]{3}','\d+ cpu -','\([\w\d]+\)','\dk ','radeon.+']
    extract_r1 = []
    for index,w in enumerate(r.lower().split()):
        if w in cpu_key:
            break
        if not any(bool(re.match(k,w)) for k in black_key) or (w == 'laptop' and index!=0):
            extract_r1.append(w)
    extract_r1 = ' '.join(extract_r1).strip(' -')
    for k in specs_key:
        extract_r1 = re.sub(k, '', extract_r1).strip(' -')
    extract_r1 = ' '.join(extract_r1.split())
    # extract_r2 = re.search(r'([\w\s-]+)',r.lower())
    # print('{0:80} {1}'.format(r,extract_r1))
    print(f'{r:70} {cleaned_row:20} {extract_r1}')
    cleaned_cellphones.Series[i] = extract_r1

cleaned_cellphones.dropna(subset=['storage_size','displ_size'],inplace=True)
cleaned_cellphones.Rating.fillna(0,inplace=True)
#recleaned displ_size
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-asus-rog-strix-g17-g713qr-hg073t.html'] = 17.3
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-hp-15-dy2093dx-405f7ua.html'] = 15.6
#lower all string attributes
lower_all_string(cleaned_cellphones)
#cpu_code fill na
for i,r in cleaned_cellphones.cpu_code[cleaned_cellphones.Brand == 'apple'].iteritems():
    if pd.isnull(r):
        for key in ['i5','i7','m1']:
            name = scraped_cellphones['item_name'][i]
            cpu = scraped_cellphones['Loại CPU'][i]
            if not isinstance(name,str):
                name = name[0]
            if not isinstance(cpu,str):
                cpu = cpu[0]
            if key in name.lower() or key in cpu.lower():
                cleaned_cellphones.cpu_code[i] = key
scraped_cellphones = pd.read_csv(data_path+"cellphones_scraped.csv",index_col='url')
# cleaned_cellphones2 = pd.read_csv("Laptop-EDA/scraped_data/cellphones_cleaned.csv",index_col='url')
cleaned_cellphones = pd.read_csv(data_path+"cellphones-cleaned.csv",index_col='url')

cleaned_cellphones.drop(columns=['1star', '2star', '3star', '4star', '5star'],inplace=True)
cleaned_cellphones.rename(columns={'comment_count':'Comment_count',
},inplace=True)
cleaned_cellphones['Price'] = cleaned_cellphones['Price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_cellphones.info()
#     cleaned_cellphones.info()
#     cleaned_cellphones2.info()
#     # print(scraped_cellphones['Series'].value_counts(dropna=False))
# cleaned_cellphones.head()

for i,cleaned_row in cleaned_cellphones.Series.iteritems():
    r = scraped_cellphones.item_name[i]
    if not isinstance(r, str):
        # print(r)
        r = r[0]
    black_key = [r'\d+gb',r'\d+inch',r'\d+gpu',r'\d+tb',
                 'laptop','chính','hãng','việt','nam','nhập','khẩu','cũ','đẹp','bạc','mqd32',
                 'apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft','huawei','avita','gigabyte']
    cpu_key = ['core','ryzen']
    specs_key = [r'\d+\sgb',r'[\d.]+\sinch',r'\d+\sgpu',r'\d+\stb',
                 'i\d [\d.]{3}','\d+ cpu -','\([\w\d]+\)','\dk ','radeon.+']
    extract_r1 = []
    for index,w in enumerate(r.lower().split()):
        if w in cpu_key:
            break
        if not any(bool(re.match(k,w)) for k in black_key) or (w == 'laptop' and index!=0):
            extract_r1.append(w)
    extract_r1 = ' '.join(extract_r1).strip(' -')
    for k in specs_key:
        extract_r1 = re.sub(k, '', extract_r1).strip(' -')
    extract_r1 = ' '.join(extract_r1.split())
    # print(f'{r:70} {cleaned_row:20} {extract_r1}')
    cleaned_cellphones.Series[i] = extract_r1

cleaned_cellphones.dropna(subset=['storage_size','displ_size'],inplace=True)
cleaned_cellphones.Rating.fillna(0,inplace=True)
#recleaned displ_size
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-asus-rog-strix-g17-g713qr-hg073t.html'] = 17.3
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-hp-15-dy2093dx-405f7ua.html'] = 15.6
#lower all string attributes
lower_all_string(cleaned_cellphones)
#cpu_code fill na
for i,r in cleaned_cellphones.cpu_code[cleaned_cellphones.Brand == 'apple'].iteritems():
    if pd.isnull(r):
        for key in ['i5','i7','m1']:
            name = scraped_cellphones['item_name'][i]
            cpu = scraped_cellphones['Loại CPU'][i]
            if not isinstance(name,str):
                name = name[0]
            if not isinstance(cpu,str):
                cpu = cpu[0]
            if key in name.lower() or key in cpu.lower():
                cleaned_cellphones.cpu_code[i] = key

cleaned_cellphones.cpu_code.loc['https://cellphones.com.vn/macbook-pro-14-inch-2021-1tb.html'] = 'm1 pro'
cleaned_cellphones.cpu_code.loc['https://cellphones.com.vn/macbook-pro-16-inch-2021-m1-max.html'] = 'm1 max'
cleaned_cellphones.OS = cleaned_cellphones.OS.replace("macos", "mac os")
cleaned_cellphones.OS = cleaned_cellphones.OS.replace("freeos", "free os")
cleaned_cellphones.OS = cleaned_cellphones.OS.replace("non-os", "free os")
#GPU
cleaned_cellphones.GPU.loc['https://cellphones.com.vn/macbook-air-2020-m1-512gb.html'] = '8 gpu'
cleaned_cellphones.GPU.loc['https://cellphones.com.vn/macbook-pro-2020-13-inch-touch-bar-m1.html'] = '8 gpu'
for i,r in cleaned_cellphones.GPU.drop_duplicates().iteritems():
    raw = scraped_cellphones['Loại card đồ họa'].loc[i]
    if not isinstance(raw, str):
        raw = raw[0]
    print(f"{r:40} {raw}")
# for c in other_cols:
#     text_facet(cleaned_cellphones,c)

NameError: name 'pd' is not defined

## Laptop Arena


In [ ]:
la_mapping_cols = ['Brand','Family','Series','Model name','Part Number',
                   'Processor manufacturer','Processor model','Internal memory',
                   'Processor series','Processor family','Release Date',
                   'Total storage capacity','Display diagonal','Maximum refresh rate']
def import_laptoparena(data_path):
    frames = []
    for file_path in glob.glob(data_path+'laptoparena*'):
        df = pd.read_csv(file_path,index_col='product_url')
        frames.append(df)
    df = pd.concat(frames)
    return df.reindex(sorted(df.columns),axis=1)

scraped_laptoparena = import_laptoparena(data_path)
print(len(scraped_laptoparena.columns))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (1,2,4,8,11,13,19,20,23,24,29,30,31,33,36,39,40,45,47,52,53,59,65,66,67,73,74,81,87,92,95,97,99,100,103,106,107,109,111,113,116,125,134,140,142,144,145,149,150,157,159,162,171,172,174,176,177,180,181,186,193,194,196,197,202,205,207,211,214,219,221,230,234,243,244,245,249,250,251,256,260,263,267,284,286,288,290,294,295,297,302,308,310,312,316,317,323,327,328,330,335,336,341,345,346,347,348,354,356,360,363,366,367,375,380,382,385,386,394,398,401,404,405,407,408) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (2,7,8,11,12,18,24,25,26,31,35,37,38,40,44,45,46,48,51,54,57,60,61,66,68,69,71,73,75,76,77,80,83,85,92,94,95,96,99,101,103,104,106,107,109,111,113,114,115,118,124,126,129,132,137,140,142,144,147,156,165,166,168,169,175,183,184,1

604


In [ ]:
print('Width: ', len(scraped_laptoparena.columns))
print('Length: ',len(scraped_laptoparena))
for c in la_mapping_cols:
    text_facet(scraped_laptoparena,c,print_all=True)

Width:  604
Length:  77464
---------- Brand ------------
DELL         15677
Acer         12097
Lenovo       11833
HP           11833
ASUS         11832
MSI           5115
Fujitsu       2092
Microsoft     1830
Toshiba       1362
Dynabook      1253
Alienware      565
Gigabyte       549
MEDION         454
Samsung        406
LG             380
Huawei         153
Sony            22
Thomson         11
Name: Brand, dtype: int64
---------- Family ------------
NaN                        7909
ThinkPad                   6349
Latitude                   5209
Aspire                     4630
Inspiron                   3605
Gaming                     3389
VivoBook                   3200
IdeaPad                    2790
Vostro                     2461
ZenBook                    2033
XPS                        1953
LIFEBOOK                   1934
ROG                        1863
Swift                      1769
ProBook                    1763
TravelMate                 1681
EliteBook                  1604


In [ ]:
def remove_duplicate(l):
    return list(dict.fromkeys(l))

def clean_laptoparena(source,use_part_number=True):
    result = pd.DataFrame()

    result['Brand'] = source['Brand'].str.strip()

    series = source[['Family','Series','Model name']]
    if not use_part_number:
        series['product_title'] = source['product_title'].str.split().str[1:].str.join(" ").str.split(' - ').str[0]
    else:
        series['product_title'] = source['product_title'].str.split().str[1:].str.join(" ")
        
    result['Series'] = series.fillna('').agg(' '.join, axis=1).str.split().apply(remove_duplicate).str.join(" ")
    text_facet(result,'Series',print_all=False)

    result['cpu_brand'] = source['Processor manufacturer'].str.strip().str.lower()
    source[['Processor family','Processor series']] = source[['Processor family','Processor series']].fillna('')
    for i,r in source[source['Processor manufacturer'].isnull()].iterrows():
        row = ' '.join(r[['Processor family','Processor series']]).lower()
        for chip in ('intel','qualcomm','amd'):
            if chip in row:
                result.cpu_brand[i] = chip

    # text_facet(source,'Processor model',print_all=False)
    result['cpu_code'] = source['Processor model'].str.split('-').str[-1].str.strip()
    # text_facet(result,'cpu_code',print_all=False)

    # text_facet(source,'Internal memory',print_all=False)
    result['ram_size'] = source['Internal memory'].str.extract(r'(\d+)',expand=False).str.strip().astype('float64')
    # text_facet(result,'ram_size',print_all=False)

    # text_facet(source,'Total storage capacity',print_all=False)
    result['storage_size'] = source['Total storage capacity'].str.extract(r'(\d+)',expand=False).str.strip().astype('float64')
    result['storage_size'][result['storage_size'] == 1000] = 1024
    # text_facet(result,'storage_size',print_all=False)

    # text_facet(source,'Display diagonal',print_all=False)
    result['displ_size'] = source['Display diagonal'].str.extract(r'([\d.]+)',expand=False).str.strip().astype('float64')
    # text_facet(result,'displ_size',print_all=False)

    # text_facet(source,'Maximum refresh rate',print_all=False)
    result['displ_rate'] = source['Maximum refresh rate'].str.extract(r'([\d.]+)',expand=False).str.strip().fillna(60).astype('int64')
    # text_facet(result,'displ_rate',print_all=False)
    result['Release_date'] = pd.to_datetime(source['Release Date'])
    lower_all_string(result)


    result.dropna(subset=mapping_cols+['Release_date'],inplace=True)
    # for c in result.columns:
    #     if len(result[result[c].isnull()]) > len(result)/2:
    #         result.drop(columns=c,inplace=True)
    #         continue
    #     else:
    #         text_facet(result,c,print_all=False)
    result = result[result.Brand.isin(all_brands)]
    return result

cleaned_laptoparena = clean_laptoparena(scraped_laptoparena,use_part_number=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


---------- Series ------------
ThinkPad X X1 Carbon                 604
IdeaPad 300 330                      538
Latitude 5000 E5470                  482
Surface Laptop 3                     457
Surface Laptop 4                     454
                                    ... 
Aspire 3 A317-52-39NP+Pack Gold17      1
14-bs096tu                             1
ENVY 13 13-ah1008nf                    1
TravelMate P2 P249-G3-M-30F8           1
TravelMate Spin B1 B118-RN-C9E9        1
Name: Series, Length: 38363, dtype: int64
Number of null: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(cleaned_laptoparena.Series)

product_url
https://www.laptoparena.net/product/50001                  10tl-231fr
https://www.laptoparena.net/product/50002                 10sdr-271fr
https://www.laptoparena.net/product/50003                 10sgs-277fr
https://www.laptoparena.net/product/50004       bravo 15 a4ddr-230xfr
https://www.laptoparena.net/product/50005    extensa 15 ex215-52-33py
                                                       ...           
https://www.laptoparena.net/product/77484              surface book 3
https://www.laptoparena.net/product/77485              surface book 3
https://www.laptoparena.net/product/77486              surface laptop
https://www.laptoparena.net/product/77487            surface laptop 3
https://www.laptoparena.net/product/77488              surface book 3
Name: Series, Length: 72334, dtype: object


In [ ]:
print(cleaned_laptoparena.cpu_code[cleaned_laptoparena.cpu_code.isnull()])

Series([], Name: cpu_code, dtype: object)


In [ ]:
with pd.option_context('display.max_columns', None):
    print(scraped_laptoparena[['Processor manufacturer','Processor series','Processor family']][scraped_laptoparena['Processor manufacturer'].isnull()])
# for c in .columns:
#     if 'processor' in c.lower():
#         text_facet(laptoparena,c)

                                          Processor manufacturer  \
product_url                                                        
https://www.laptoparena.net/product/50571                    NaN   
https://www.laptoparena.net/product/50764                    NaN   
https://www.laptoparena.net/product/51115                    NaN   
https://www.laptoparena.net/product/51155                    NaN   
https://www.laptoparena.net/product/51604                    NaN   
...                                                          ...   
https://www.laptoparena.net/product/72033                    NaN   
https://www.laptoparena.net/product/72034                    NaN   
https://www.laptoparena.net/product/72035                    NaN   
https://www.laptoparena.net/product/77092                    NaN   
https://www.laptoparena.net/product/77264                    NaN   

                                          Processor series  \
product_url                                          

# Integrate
cleaned data are saved as `cleaned_tgdd`,`cleaned_fpt`,`cleaned_cellphones`

TODO
- Chọn các attribute và xử lý các attribute đó trong cả 3 file - Series ✅
- Kết nối 3 data của tgdd, cellphone và fpt bao gồm các mapping attribute và hợp nhất các retained attribute ✅ (cách hợp nhất các retained phải xem lại)
- Kết nối data thu được ở bước 2 với data của laptop arena dựa vào các mapping attribute, giữ lại các retained.
- Xử lý data đã map được ở bước 3 (loại bỏ các cột quá nhiều null, format dữ liệu, điền tay cho các máy của apple)


## Check out mapping attribute in TGDD, FPT and CellphoneS
- Mapping attribute must be non-null, except for Apple brand ✅
- Retained attribute type is float or int ✅
- Three dataset must have same set of column ✅
- No duplicated rows ✅

In [ ]:
cleaned_fpt[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size',
             'displ_size','displ_rate',
             'Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_fpt.Brand[cleaned_fpt.cpu_code.isnull()])
    print(cleaned_fpt[cleaned_fpt['P/N'].isnull()])
cleaned_fpt.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series         146 non-null    object 
 1   Brand          146 non-null    object 
 2   cpu_brand      146 non-null    object 
 3   cpu_code       146 non-null    object 
 4   ram_size       146 non-null    int64  
 5   storage_size   146 non-null    int64  
 6   displ_size     146 non-null    float64
 7   displ_rate     146 non-null    int64  
 8   Price          146 non-null    int64  
 9   Rating         143 non-null    float64
 10  Rating_count   146 non-null    int64  
 11  Comment_count  143 non-null    float64
dtypes: float64(3), int64(5), object(4)
memory usage: 13.8+ KB
Series([], Name: Brand, dtype: object)
Empty DataFrame
Columns: [P/N, Rating, Rating_count, Comment_count, Brand, Series, Price, cpu_brand, cpu_code, ram_size, ram_upgradable, GPU, Battery, storage_type, storage_size

,P/N,Rating,Rating_count,Comment_count,Brand,Series,Price,cpu_brand,cpu_code,ram_size,ram_upgradable,GPU,Battery,storage_type,storage_size,storage_extra_slot,displ_size,displ_resolution,displ_rate,screen_technology,Weight,length,width,thickness,OS,Surface,Release_year,hdmi,headphone_jack,lan,type_c,usb_a,sd_card,backlit_keyboard
0,nh.qd9sv.001,0.0,0,598.0,acer,nitro gaming an515-57-74nu nh.qd9sv.001,28499000,intel,11800h,8,yes,nvidia geforce rtx 3050ti,NaN,ssd,512,yes,15.6,1920 x 1080,144,led,2.20,363.4,255.0,23.9,windows 10,plastic,2021,yes,yes,yes,yes,yes,no,yes
1,852vn,0.0,0,340.0,msi,modern 14 b11mou 852vn,18799000,intel,1155g7,8,yes,intel iris xe graphics,65.0,ssd,512,yes,14.0,1920 x 1080,60,ips,1.30,319.0,219.0,16.9,windows 10,metal,2021,yes,yes,yes,yes,yes,yes,yes
2,p112f001,0.0,0,116.0,dell,inspiron n3511 p112f001,15299000,intel,1115g4,4,yes,intel uhd graphics,65.0,ssd,256,yes,15.6,1920 x 1080,60,va,1.73,358.5,258.0,18.9,windows 10,plastic,2021,yes,yes,yes,yes,yes,yes,yes


In [ ]:
cleaned_cellphones[all_cols].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_cellphones.Brand[cleaned_cellphones.Price.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.ram_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.storage_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.displ_size.isnull()])
cleaned_cellphones.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html to https://cellphones.com.vn/surface-pro-7-plus.html
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Comment_count       416 non-null    int64  
 1   Price               352 non-null    float64
 2   Rating              416 non-null    float64
 3   Rating_count        416 non-null    int64  
 4   Series              416 non-null    object 
 5   Brand               416 non-null    object 
 6   cpu_brand           416 non-null    object 
 7   cpu_code            416 non-null    object 
 8   ram_size            416 non-null    float64
 9   storage_size        416 non-null    float64
 10  displ_size          416 non-null    float64
 11  displ_rate          416 non-null    int64  
 12  Battery             364 non-null    float64
 13  GPU                 413 non-null   

,sd_card,P/N,ram_upgradable,GPU,displ_rate,backlit_keyboard,Weight,thickness,width,length,screen_technology,Battery,OS,displ_resolution,Surface,displ_size,Price,Comment_count,Rating_count,Rating,ram_size,cpu_code,cpu_brand,storage_size,storage_type,storage_extra_slot,Release_year,Brand,Series,usb_a,type_c,lan,headphone_jack,hdmi
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html,yes,mqd32,NaN,intel hd 6000,60,NaN,1.35,17,227.0,325.00,NaN,54.0,mac os,1440 x 900,NaN,13.3,20000000.0,306,0,0.0,8.0,i5,intel,128.0,ssd,NaN,2017.0,apple,macbook air i,yes,no,no,no,no
https://cellphones.com.vn/aptop-dell-inspiron-15-3505-y1n1t2.html,no,y1n1t2,yes,amd radeon,60,no,1.83,18,249.0,363.96,NaN,42.0,windows 10,1920 x 1080,NaN,15.6,18290000.0,28,0,0.0,8.0,3500u,amd,512.0,ssd,NaN,NaN,dell,inspiron 3505 y1n1t2,yes,no,yes,yes,yes
https://cellphones.com.vn/huawei-matebook-14.html,no,NaN,NaN,nvidia geforce mx350,60,NaN,1.53,15.9,223.8,307.50,NaN,56.0,windows 10,2160 x 1440,NaN,14.0,NaN,3,0,0.0,8.0,10210u,intel,512.0,ssd,NaN,NaN,huawei,matebook 14,no,yes,no,yes,yes


In [ ]:
cleaned_tgdd[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size','displ_size','displ_rate','Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_tgdd.Brand[cleaned_tgdd.cpu_code.isnull()])
cleaned_tgdd.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 181 entries, https://www.thegioididong.com/laptop/msi-gf66-11uc-i7-224vn?src=osp#top-tskt to https://www.thegioididong.com/laptop/acer-aspire-a315-56-308n-i3-nxhs5sv00c?src=osp#top-tskt
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series         181 non-null    object 
 1   Brand          181 non-null    object 
 2   cpu_brand      181 non-null    object 
 3   cpu_code       181 non-null    object 
 4   ram_size       181 non-null    float64
 5   storage_size   181 non-null    float64
 6   displ_size     181 non-null    float64
 7   displ_rate     181 non-null    int64  
 8   Price          181 non-null    float64
 9   Rating         181 non-null    float64
 10  Rating_count   181 non-null    int64  
 11  Comment_count  181 non-null    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 23.4+ KB
Series([], Name: Brand, dtype: object)


,P/N,Rating,Rating_count,Comment_count,Brand,Series,Price,cpu_brand,cpu_code,ram_size,ram_upgradable,GPU,Battery,storage_size,storage_type,storage_extra_slot,displ_size,displ_resolution,displ_rate,screen_technology,Weight,length,width,thickness,OS,Surface,Release_year,backlit_keyboard,hdmi,headphone_jack,lan,type_c,usb_a,sd_card
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://www.thegioididong.com/laptop/msi-gf66-11uc-i7-224vn?src=osp#top-tskt,224vn,3.0,5,70,msi,katana gaming gf66 11uc 224vn,29990000.0,intel,11800h,8.0,yes,nvidia geforce rtx3050,53.0,512.0,ssd,no,15.6,1920 x 1080,144,ips,2.10,359.0,259.0,24.90,windows 10,plastic,2021.0,yes,yes,yes,yes,yes,yes,no
https://www.thegioididong.com/laptop/msi-gaming-pulse-gl66-11udk-i7-816vn?src=osp#top-tskt,816vn,0.0,0,13,msi,gaming pulse gl66 11udk 816vn,33990000.0,intel,11800h,16.0,yes,nvidia geforce rtx3050ti,53.5,512.0,ssd,no,15.6,1920 x 1080,144,ips,2.25,359.0,259.0,23.95,windows 10,metal,2021.0,yes,yes,no,yes,yes,yes,no
https://www.thegioididong.com/laptop/acer-nitro-gaming-an515-57-727j-i7-nhqd9sv005?src=osp#top-tskt,nh.qd9sv.005,3.6,13,220,acer,nitro 5 gaming an515 57 727j nh.qd9sv.005,29990000.0,intel,11800h,8.0,yes,nvidia geforce rtx3050ti,57.0,512.0,ssd,yes,15.6,1920 x 1080,144,ips,2.20,363.4,255.0,23.90,windows 10,plastic,2021.0,yes,yes,yes,yes,yes,yes,no


In [ ]:
print(sorted(cleaned_fpt.columns))
print(sorted(cleaned_tgdd.columns))
print(sorted(cleaned_cellphones.columns))

['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology', 'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 'thickness', 'type_c', 'usb_a', 'width']
['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology', 'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 'thickness', 'type_c', 'usb_a', 'width']
['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Wei

In [ ]:
print(f"Duplicated tgdd: {any(cleaned_tgdd.duplicated())}")
print(f"Duplicated cellphones: {any(cleaned_cellphones.duplicated())}")
print(f"Duplicated fpt: {any(cleaned_fpt.duplicated())}")

Duplicated tgdd: False
Duplicated cellphones: False
Duplicated fpt: False


Rating_count chia làm 2 tập

In [ ]:
# col = 'Rating'
# col = 'Release_year'
# col = 'Comment_count'
# col = 'Brand'
# col = 'storage_type'
for col in mapping_cols:
    for data in (cleaned_cellphones,cleaned_fpt,cleaned_tgdd):
        text_facet(data[data.Brand == 'apple'],col)
# data = cleaned_tgdd
# data[data.Brand == 'apple'].Series

---------- Series ------------
imac 24 2021 m1 i                     5
macbook air m1 2020 i                 4
macbook pro 13 touch bar m1 2020 i    4
imac 27 2020                          3
macbook pro 13 touch bar 2020 i       3
macbook pro 2021                      2
imac 21.5 2020                        2
macbook air i                         1
macbook pro m1 pro 16 2021            1
imac 24 2021 m1                       1
macbook pro 14 m1 pro 2021            1
macbook pro 16 m1 max 2021            1
macbook pro 16 touch bar 2019         1
Name: Series, dtype: int64
---------- Series ------------
macbook pro 2020 touch bar m1    2
macbook pro 2021 m1 pro          2
macbook pro 2021 m1 max          1
macbook pro 2020 touch bar       1
macbook air 2020 m1              1
Name: Series, dtype: int64
---------- Series ------------
macbook pro m1 2020    4
macbook air m1 2020    3
Name: Series, dtype: int64
---------- Brand ------------
apple    29
Name: Brand, dtype: int64
---------- Br

## Merge TGDD, FPT and CellphoneS
Retained attributes:
- Comment = Sum of 3
- Rating Count = Sum of 3
- Availabe_in = Number of existence in stores
- Rating point = Average of `Rating` with weight is `Rating_count`
- Price = Average of `Price`

String similarity
<ol>
<li><a href="http://en.wikipedia.org/wiki/Hamming_distance">Hamming distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Levenshtein_distance">Levenshtein distance</a></li>
<li><a href="http://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance">Damerau–Levenshtein distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance">Jaro–Winkler distance</a></li>
</ol>

Jellyfish: https://github.com/jamesturk/jellyfish

In [ ]:
key_word = []
black_key = scraped_laptoparena['Market positioning'].dropna().str.lower().unique().tolist()
for c in ['Family','Series','Model name']:
    key_word.extend(scraped_laptoparena[c].fillna('').str.lower().tolist())
key_word.extend(['macbook','air','pro','katana','slim','duet','firefly','nano','x1','liber','victus','pulse','raider','carbon'])
key_word = list(set(" ".join(key_word).split()))
key_word = [k.strip('?-)(, ') for k in key_word if len(k.strip('?-)(, '))>=3 and '-' not in k and not re.search(r'\d+',k)]
for k in black_key:
    try: key_word.remove(k)
    except: continue

In [ ]:
'special edition' in key_word

False

In [ ]:
cleaned_laptoparena.Series["https://www.laptoparena.net/product/75030"]

'latitude 3000 3520'

In [ ]:
lev = textdistance.Levenshtein(qval=None)
hamming =  textdistance.Hamming(qval=None)
def similar_str(a, b, algo):
    '''
    Compare string similarity with threshold
    Input: a,b - string, thres - float 
    Output: float
    '''
    if algo == 'normal':
        return SequenceMatcher(None, a, b).ratio()

    elif algo == 'custom':
        both = 0
        a_word, b_word = 0,0
        just_a, just_b = [],[]
        for x in a.split():
            if x in b.split() and x in key_word: both += 1
            if x not in key_word or x not in b.split(): just_a.append(x)
            if x in key_word: a_word += 1
        for x in b.split():
            if x in key_word: b_word += 1
            if x not in a.split() or x not in key_word: just_b.append(x)
        # token_rate = 2*both/max(a_word+b_word,1)
        token_rate = max(both/max(a_word,1),both/max(b_word,1))
        # print('token: ',token_rate)
        if token_rate == 0:
            return fuzz.ratio(" ".join(just_a)," ".join(just_b))/100
        else:
            return 0.7*token_rate + 0.3*fuzz.ratio(" ".join(just_a)," ".join(just_b))/100

    elif algo == 'lev2':
        return lev.normalized_similarity(a,b)
    elif algo == 'lev':
        return fuzz.ratio(a,b)/100

s = [('gaming predator helios 300 ph315-54-78w5 nh.qc5sv.001','Predator Helios 300 PH315-53-781R NH.Q7YAA.001'.lower()),
    ('Nitro AN515-51-79DZ'.lower(), 'nitro 5 an515 57 74nu nh.qd9sv.00 1'),
    ('macbook air 2020 m1','macbook pro m1 2020'),
    ('gaming stealth15m a11sdk 061vn','gaming stealth 15m a11uek 254vn'),
    ('yoga duet 7 13iml05 82as007bvn','yoga 6 13alc6'),
    ('latitude 3520 70251593','latitude 3000 3520')
]
# for i in ('Nitro','5','AN515-51-79DZ','NH.Q2QAA.007'):
#     print(similar_str(i.lower(),s3[1],'lev'))
#     print(similar_str(i.lower(),s3[1],'custom'))
#     print('-----------------------------------------')
for a,b in s:
    print(similar_str(a,b,'lev'))
    print(similar_str(a,b,'custom'))
    print('-----------------------------------------')

0.79
0.9069999999999999
-----------------------------------------
0.52
0.814
-----------------------------------------
0.74
0.515
-----------------------------------------
0.82
0.82
-----------------------------------------
0.42
0.7719999999999999
-----------------------------------------
0.65
0.8079999999999999
-----------------------------------------


In [ ]:
def merge(dfa,dfb,pref1='_df1',pref2='_df2',laptoparena_position=None,
          min_score = 0.55, algo = 'lev'):

    df1 = dfa.copy()
    df2 = dfb.copy()

    df1['Index'+pref1]=df1.index
    df2['Index'+pref2]=df2.index

    df1['match_index'+pref2] = np.nan
    df1['match_score'+pref2] = np.nan
    
    choices = []
    for i1,r1 in df1.iterrows():
        score = df2['Series'].apply(similar_str,args=(r1['Series'],algo))
        match_df2 = df2.loc[
            (df2['ram_size'] == r1['ram_size']) &
            (df2['cpu_brand'] == r1['cpu_brand']) &
            (df2['cpu_code'] == r1['cpu_code']) &
            (df2['Brand'] == r1['Brand']) &
            (df2['storage_size'] == r1['storage_size']) &
            (df2['displ_rate']  == r1['displ_rate']) &
            (abs(r1['displ_size']-df2['displ_size'])<0.5) &
            (score > min_score)
        ]
        choices.extend(list(zip(*[[i1]*len(match_df2),match_df2.index,score.loc[match_df2.index]])))

    exclude1, exclude2 = [], []
    for i1,i2,score in sorted(choices,key=lambda x:x[2],reverse=True):
        if i1 not in exclude1 and i2 not in exclude2:
            df1['match_index'+pref2][i1] = i2
            df1['match_score'+pref2][i1] = score
            exclude1.append(i1)
            exclude2.append(i2)

    print(f"Matched: {len(df1['match_index'+pref2][np.invert(df1['match_index'+pref2].isnull())])}")

    merged = pd.merge(df1, df2, suffixes=[pref1,pref2], 
                      left_on='match_index'+pref2, right_index = True, 
                      how = laptoparena_position if bool(laptoparena_position) else 'outer'
                      )
    merged.reset_index(inplace=True)
    return merged.reindex(sorted(merged.columns), axis=1)

def join_column_from_merged(df, pref1, pref2,priority='left',keep_cols=[]):
    merged_retained = []
    for c in df.columns:
        # print(c)
        if any(k+'_' in c for k in retained_cols + keep_cols):
            merged_retained.append(c)
    print(merged_retained)
    result = df[merged_retained]

    # to_merge = mapping_cols + other_cols
    to_merge = all_cols
    for c in to_merge:
        result[c] = np.nan
        for i,r in result.iterrows():
            if pd.isnull(df[c+pref1][i]):
                result[c][i] = df[c+pref2][i]
            elif pd.isnull(df[c+pref2][i]):
                result[c][i] = df[c+pref1][i]
            elif priority == 'left':
                result[c][i] = df[c+pref1][i]
            elif priority == 'right':
                result[c][i] = df[c+pref2][i]
    return result

def join_retained_atrr(dataframe, prefs):
    df = dataframe.copy()
    df['Availabe_in'] = len(prefs) - df[['Index'+p for p in prefs]].isnull().sum(axis=1)
    for i,r in df.iterrows():
        r.Rating_count = np.sum(r[['Rating_count'+p for p in prefs]])
        if r.Rating_count ==0:
            r.Rating = 0
        else:
            r.Rating = np.average(r[['Rating'+p for p in prefs]].fillna(0),
                                weights = r[['Rating_count'+p for p in prefs]].fillna(0))
        r.Comment_count = np.sum(r[['Comment_count'+p for p in prefs]])
        r.Price = np.mean(r[['Price'+p for p in prefs]])
        df.loc[i] = r

    for c in retained_cols:
        with pd.option_context('display.max_columns', None):
            print(df[[c]+[c+p for p in prefs]])

    for c in df.columns:
        for p in prefs:
            if any(k+p in c for k in retained_cols):
                df = df.drop(columns=c)
    return df

In [ ]:
df = merge(cleaned_fpt, cleaned_tgdd,'_fpt','_tgdd',algo='custom')
# df = merge(cleaned_tgdd, cleaned_fpt,'_tgdd','_fpt')
df_2 = join_column_from_merged(df,'_fpt','_tgdd',priority='right',
                               keep_cols=['Index','Series','match_score'])
df_3 = merge(df_2,cleaned_cellphones,'_fpt_tgdd','_cellphones',algo='custom')
df_4 = join_column_from_merged(df_3,'_fpt_tgdd','_cellphones',priority='left',keep_cols=['Index'])
df_4.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57:

Matched: 61
['Comment_count_fpt', 'Comment_count_tgdd', 'Index_fpt', 'Index_tgdd', 'Price_fpt', 'Price_tgdd', 'Rating_count_fpt', 'Rating_count_tgdd', 'Rating_fpt', 'Rating_tgdd', 'Series_fpt', 'Series_tgdd', 'match_score_tgdd']


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[key] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages

Matched: 130
['Comment_count_cellphones', 'Comment_count_fpt', 'Comment_count_fpt_tgdd', 'Comment_count_tgdd', 'Index_cellphones', 'Index_fpt', 'Index_fpt_tgdd', 'Index_tgdd', 'Price_cellphones', 'Price_fpt', 'Price_fpt_tgdd', 'Price_tgdd', 'Rating_cellphones', 'Rating_count_cellphones', 'Rating_count_fpt', 'Rating_count_fpt_tgdd', 'Rating_count_tgdd', 'Rating_fpt', 'Rating_fpt_tgdd', 'Rating_tgdd']


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[key] = value


,Comment_count_cellphones,Comment_count_fpt,Comment_count_fpt_tgdd,Comment_count_tgdd,Index_cellphones,Index_fpt,Index_fpt_tgdd,Index_tgdd,Price_cellphones,Price_fpt,Price_fpt_tgdd,Price_tgdd,Rating_cellphones,Rating_count_cellphones,Rating_count_fpt,Rating_count_fpt_tgdd,Rating_count_tgdd,Rating_fpt,Rating_fpt_tgdd,Rating_tgdd,Comment_count,Price,Rating,Rating_count,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,GPU,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width
0,44.0,598.0,264.0,264.0,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,0.0,https://www.thegioididong.com/laptop/acer-nitr...,27390000.0,28499000.0,29990000.0,29990000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264.0,29990000.0,0.0,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57.0,nvidia geforce rtx3050ti,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021.0,plastic,2.20,yes,yes,363.40,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255.00
1,7.0,340.0,53.0,53.0,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,1.0,https://www.thegioididong.com/laptop/msi-gamin...,21990000.0,18799000.0,22990000.0,22990000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,22990000.0,0.0,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39.0,nvidia geforce mx450,windows 10,669vn,1920 x 1080,yes,2021.0,metal,1.30,yes,no,319.00,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219.00
2,NaN,116.0,206.0,206.0,NaN,2.0,2.0,https://www.thegioididong.com/laptop/dell-insp...,NaN,15299000.0,15290000.0,15290000.0,NaN,NaN,0.0,18.0,18.0,0.0,3.7,3.7,206.0,15290000.0,3.7,18.0,inspiron 15 3511 p112f001abl,dell,intel,1115g4,4.0,256.0,15.6,60.0,41.0,intel uhd graphics,windows 10,p112f001abl,1920 x 1080,yes,2021.0,plastic,1.70,no,no,358.50,yes,va,yes,yes,ssd,yes,18.9,no,yes,235.50
3,NaN,378.0,378.0,NaN,NaN,4.0,4.0,NaN,NaN,14999000.0,14999000.0,NaN,NaN,NaN,4.0,4.0,NaN,5.0,5.0,NaN,378.0,14999000.0,5.0,4.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,NaN,intel uhd graphics,windows 10,82h80040vn,1920 x 1080,yes,2021.0,plastic,1.65,yes,yes,359.00,yes,led,yes,yes,ssd,yes,19.9,yes,yes,237.70
4,NaN,1244.0,1244.0,NaN,NaN,6.0,5.0,NaN,NaN,18799000.0,18799000.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,1244.0,18799000.0,5.0,5.0,ideapad slim 5 15itl05 82fg00r0vn,lenovo,intel,1135g7,16.0,512.0,15.6,60.0,45.0,intel iris xe graphics,windows 10,82fg00r0vn,1920 x 1080,yes,2021.0,"metal, plastic",1.66,yes,yes,356.67,yes,ips,yes,yes,ssd,yes,17.9,yes,yes,233.13


In [ ]:
def print_merged_result(df, col, prefs=('_fpt','_tgdd','_cellphones'),sort=True):
    '''prefs can be any length'''
    print(f"-------Column: {col} ---------------")
    if sort:
        df = df.iloc[df[[col+p for p in prefs]].fillna('').agg(''.join, axis=1).argsort()]
    for i, r in df.iterrows():
        # if abs(r[col+pref1]-r[col+pref2]) >=0.5:
        for p in prefs[1:]:
            print(f"{r['match_score'+p]:4.2f} |",end=" ")
        for p in prefs:
            print(f"{r[col+p]:45} |",end=" ")
        for p in prefs:
            print(f"{r['Index'+p]:80} |",end=" ")
        print("")
# {r[col+pref1]:45} | {r[col+pref2]:40} | {r[col+pref3]:40} | {r['Index'+pref1]:10} | {r['Index'+pref2]:80} | {r['Index'+pref3]:80}")


# print_merged_result(df,col='Series')
print_merged_result(df_3,col='Series',sort=True)


-------Column: Series ---------------
 nan |  nan |                                           nan |                                           nan | 14-cf2033wm 3v7g4ua                           |                                                                              nan |                                                                              nan | https://cellphones.com.vn/laptop-hp-14-cf2033wm-3v7g4ua-pent.html                | 
 nan |  nan |                                           nan |                                           nan | 14-dq2055wm 39k15ua                           |                                                                              nan |                                                                              nan | https://cellphones.com.vn/laptop-hp-14-dq2055wm-39k15ua.html                     | 
 nan |  nan |                                           nan |                                           nan | 14s-dq2544tu 46m22pa                

In [ ]:
df_5 = join_retained_atrr(df_4,('_fpt','_tgdd','_cellphones'))
df_5.head()

     Comment_count  Comment_count_fpt  Comment_count_tgdd  \
0            906.0              598.0               264.0   
1            400.0              340.0                53.0   
2            322.0              116.0               206.0   
3            378.0              378.0                 NaN   
4           1244.0             1244.0                 NaN   
..             ...                ...                 ...   
547           39.0                NaN                 NaN   
548           21.0                NaN                 NaN   
549           76.0                NaN                 NaN   
550           49.0                NaN                 NaN   
551           29.0                NaN                 NaN   

     Comment_count_cellphones  
0                        44.0  
1                         7.0  
2                         NaN  
3                         NaN  
4                         NaN  
..                        ...  
547                      39.0  
548        

,Index_cellphones,Index_fpt,Index_fpt_tgdd,Index_tgdd,Comment_count,Price,Rating,Rating_count,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,GPU,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width,Availabe_in
0,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,0.0,https://www.thegioididong.com/laptop/acer-nitr...,906.0,2.862633e+07,0.0,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57.0,nvidia geforce rtx3050ti,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021.0,plastic,2.20,yes,yes,363.40,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255.00,3
1,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,1.0,https://www.thegioididong.com/laptop/msi-gamin...,400.0,2.125967e+07,0.0,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39.0,nvidia geforce mx450,windows 10,669vn,1920 x 1080,yes,2021.0,metal,1.30,yes,no,319.00,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219.00,3
2,NaN,2.0,2.0,https://www.thegioididong.com/laptop/dell-insp...,322.0,1.529450e+07,3.7,18.0,inspiron 15 3511 p112f001abl,dell,intel,1115g4,4.0,256.0,15.6,60.0,41.0,intel uhd graphics,windows 10,p112f001abl,1920 x 1080,yes,2021.0,plastic,1.70,no,no,358.50,yes,va,yes,yes,ssd,yes,18.9,no,yes,235.50,2
3,NaN,4.0,4.0,NaN,378.0,1.499900e+07,5.0,4.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,NaN,intel uhd graphics,windows 10,82h80040vn,1920 x 1080,yes,2021.0,plastic,1.65,yes,yes,359.00,yes,led,yes,yes,ssd,yes,19.9,yes,yes,237.70,1
4,NaN,6.0,5.0,NaN,1244.0,1.879900e+07,5.0,5.0,ideapad slim 5 15itl05 82fg00r0vn,lenovo,intel,1135g7,16.0,512.0,15.6,60.0,45.0,intel iris xe graphics,windows 10,82fg00r0vn,1920 x 1080,yes,2021.0,"metal, plastic",1.66,yes,yes,356.67,yes,ips,yes,yes,ssd,yes,17.9,yes,yes,233.13,1


## Recleaned merged data
 - Recleaned `GPU`

In [ ]:
text_facet(scraped_cellphones,'Loại card đồ họa',print_all=True)
text_facet(cleaned_cellphones,'GPU',print_all=True)


---------- Loại card đồ họa ------------
Intel Iris Xe Graphics                                                                               95
Intel UHD Graphics                                                                                   66
Intel® Iris® Plus Graphics                                                                           43
AMD Radeon™ Graphics                                                                                 28
NVIDIA GeForce GTX 1650 4GB GDDR6                                                                    26
GPU 8 nhân, 16 nhân Neural Engine                                                                    24
AMD Radeon Graphics                                                                                  17
NVIDIA GeForce RTX 3050 4GB GDDR6                                                                    16
GPU 8 nhân                                                                                           16
Intel® Iris® Xe Graphic

In [ ]:
integrated_gpu_la = scraped_laptoparena['On-board graphics adapter model'].drop_duplicates().dropna()
discrete_gpu_la = scraped_laptoparena['Discrete graphics adapter model'].drop_duplicates(keep='last').dropna()
gpu_laptoparena = integrated_gpu_la.append(discrete_gpu_la).rename('Value_laptoparena').str.lower().str.strip()

def merge_series(dfa,dfb,pref1='_df1',pref2='_df2', min_score = 0.55, algo = 'lev'):
    '''dfa and dfb is series'''
    merged = pd.DataFrame()
    merged['Value'+pref1] = dfa.fillna('')
    merged['Index'+pref1]=dfa.index
    s2 = dfb.rename('Value'+pref2).fillna('')
    def fmatch_index(r1):
        '''
        apply for df1 only
        input: r1 - row in dataframe df1
        return the index of matched row in df2 with highest Series similarity
        '''
        choices = []
        for i,r2 in s2.iteritems():
            score = similar_str(r1['Value'+pref1],r2,algo)
            if score > min_score:
                choices.append((i,score))
        return max(choices, key=lambda x:x[1]) if choices else None

    merged[['df2_matchindex','match_score']] = merged.apply(fmatch_index,axis=1,result_type='expand')
    print(f"Matched: {len(merged['df2_matchindex'][np.invert(merged['df2_matchindex'].isnull())])}")
    # merged['Value'+pref2] = 
    merged = pd.merge(merged, s2,
                    left_on='df2_matchindex',
                    right_index = True,
                    suffixes=[pref1,pref2],
                    how = 'left') 
    return merged


print(len(df_5.GPU))
pref1,pref2 = '_vn','_laptoparena'
merged = merge_series(df_5.GPU,gpu_laptoparena,pref1=pref1,pref2=pref2)

def fix(df, df1_value, change_value):
    df['Value'+pref2] = np.where(df['Value'+pref1].fillna('').str.strip() == df1_value, 
                                 change_value, df['Value'+pref2])

fix(merged, 'nvidia geforce  gtx 3050','nvidia geforce rtx 3050')
fix(merged, 'amd radeon rx 6600m','amd radeon rx 6600m')
fix(merged, 'amd radeon rx graphics','amd radeon rx graphics')
fix(merged, 'radeon rx graphics','amd radeon rx graphics')
fix(merged, 'amd radeon vega 8 graphics','amd radeon vega 8')
fix(merged, 'intel hd 6000','intel hd graphics 6000')
fix(merged, 'amd radeon pro 5300','amd radeon pro 5300')
fix(merged, 'amd radeon pro 5500 xt','amd radeon pro 5500 xt')
fix(merged, 'intel\xa0uhd','intel uhd graphics')
fix(merged, 'intel iris xe','intel iris xe graphics')
fix(merged, 'nvidia geforce gtx 1650 + amd radeon','nvidia geforce gtx 1650')
fix(merged, 'nvidia geforce gtx 1650ti + amd radeon rx vega 8','nvidia geforce gtx 1650 ti')
fix(merged, 'nvidia geforce gtx 1650ti + amd radeon','nvidia geforce gtx 1650 ti')
fix(merged, 'nvidia geforce gtx 1660ti + amd radeon','nvidia geforce gtx 1660 ti')
fix(merged, 'intel uhd g1','intel uhd graphics g1')
fix(merged, 'amd radeon rx vega','amd radeon graphics')
fix(merged, 'integrated intel uhd','intel uhd graphics')
fix(merged, 'integrated amd radeon','amd radeon graphics')
fix(merged, 'amd radeon pro 5300m','amd radeon pro 5300m')
fix(merged, '8 nhân gpu','m1 8-core gpu')
fix(merged, '7 nhân gpu','m1 7-core gpu')
fix(merged, '7 gpu','m1 7-core gpu')
fix(merged, '8 gpu','m1 8-core gpu')
fix(merged, 'apple m1','m1 8-core gpu')
fix(merged, '14 gpu','M1 Pro 14-Core GPU'.lower())
fix(merged, '16 gpu','M1 Pro 16-Core GPU'.lower())
fix(merged, 'apple m1 pro','M1 Pro 16-Core GPU'.lower())
fix(merged, '32 gpu','M1 Max 32-Core GPU'.lower())
fix(merged, 'apple m1 max','M1 Max 32-Core GPU'.lower())
fix(merged, 'amd radeon','amd radeon graphics')

for i,r in merged['Value'+pref1].drop_duplicates().iteritems():
    if merged['match_score'].loc[i] != 1:
        print(f"{merged['match_score'].loc[i]:4} {r:40} {merged['Value'+pref2].loc[i]:40} {i}")

df_6 = merged = pd.merge(merged[['Index'+pref1,'Value'+pref2]], df_5,
                    left_on='Index_vn',
                    right_index = True,
                    how = 'right')
 
df_6.drop(columns=['Index'+pref1,'GPU','Index_fpt_tgdd'],inplace=True)
df_6.rename(columns={'Value'+pref2:'GPU'},inplace=True)
# print(len(cleaned_fpt.GPU))
# fpt_gpu = merge_series(cleaned_fpt.GPU,gpu_laptoparena)
# fpt_gpu[fpt_gpu.Value_laptoparena.isnull()]
# print()
# , integrated intel uhd, 
# amd radeon rx vega, 

# intel uhd g1

552
Matched: 526
0.96 nvidia geforce rtx3050ti                 nvidia geforce rtx 3050 ti               0
0.82 amd radeon rx graphics vega 7            amd radeon rx vega 7                     5
0.92 nvidia geforce  gtx 3050                 nvidia geforce rtx 3050                  7
0.98 nvidia geforce rtx 3050                  nvidia geforce rtx 3050                  12
0.92 amd radeon rx 6600m                      amd radeon rx 6600m                      13
0.98 nvidia geforce rtx 3060 max-q            nvidia geforce rtx 3060 max-q            14
0.96 nvidia geforce rtx3050                   nvidia geforce rtx 3050                  15
 0.8 geforce gtx 1650                         nvidia geforce gtx 1650                  16
 nan apple m1 max                             m1 max 32-core gpu                       17
 nan apple m1 pro                             m1 pro 16-core gpu                       18
0.84 iris xe graphics                         intel iris xe graphics                  

In [ ]:
text_facet(df_6,'GPU',print_all=True,ascending=True)

---------- GPU ------------
nvidia geforce rtx 2070               1
amd radeon rx 5600m                   1
nvidia quadro t500                    1
nvidia geforce gtx 1050               1
intel hd graphics 6000                1
intel uhd graphics g1                 1
m1 pro 14-core gpu                    1
intel iris graphics                   1
amd radeon pro 560x                   1
nvidia geforce rtx 3070 max-q         1
amd radeon rx 6600m                   1
intel iris xe max                     1
amd radeon pro 5300m                  1
amd radeon rx vega 6                  1
amd radeon rx vega 8                  1
amd radeon rx vega 7                  1
amd radeon pro 5500 xt                1
amd radeon vega 8                     2
nvidia geforce gtx 1660 ti max-q      2
amd radeon pro 5300                   2
amd radeon rx 5300m                   2
nvidia geforce mx250                  2
m1 max 32-core gpu                    2
nvidia geforce rtx 2070 max-q         2
amd radeon r

In [ ]:
df_6

,GPU,Index_cellphones,Index_fpt,Index_tgdd,Comment_count,Price,Rating,Rating_count,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width,Availabe_in
0,nvidia geforce rtx 3050 ti,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,https://www.thegioididong.com/laptop/acer-nitr...,906.0,2.862633e+07,0.0,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57.0,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021.0,plastic,2.20,yes,yes,363.40,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255.00,3
1,nvidia geforce mx450,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,https://www.thegioididong.com/laptop/msi-gamin...,400.0,2.125967e+07,0.0,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39.0,windows 10,669vn,1920 x 1080,yes,2021.0,metal,1.30,yes,no,319.00,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219.00,3
2,intel uhd graphics,NaN,2.0,https://www.thegioididong.com/laptop/dell-insp...,322.0,1.529450e+07,3.7,18.0,inspiron 15 3511 p112f001abl,dell,intel,1115g4,4.0,256.0,15.6,60.0,41.0,windows 10,p112f001abl,1920 x 1080,yes,2021.0,plastic,1.70,no,no,358.50,yes,va,yes,yes,ssd,yes,18.9,no,yes,235.50,2
3,intel uhd graphics,NaN,4.0,NaN,378.0,1.499900e+07,5.0,4.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,NaN,windows 10,82h80040vn,1920 x 1080,yes,2021.0,plastic,1.65,yes,yes,359.00,yes,led,yes,yes,ssd,yes,19.9,yes,yes,237.70,1
4,intel iris xe graphics,NaN,6.0,NaN,1244.0,1.879900e+07,5.0,5.0,ideapad slim 5 15itl05 82fg00r0vn,lenovo,intel,1135g7,16.0,512.0,15.6,60.0,45.0,windows 10,82fg00r0vn,1920 x 1080,yes,2021.0,"metal, plastic",1.66,yes,yes,356.67,yes,ips,yes,yes,ssd,yes,17.9,yes,yes,233.13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,amd radeon graphics,https://cellphones.com.vn/surface-laptop-4.html,NaN,NaN,39.0,2.699000e+07,0.0,0.0,surface laptop 4,microsoft,amd,4680u,8.0,256.0,13.5,60.0,47.0,windows 10,NaN,2256 x 1504,no,NaN,NaN,1.27,NaN,no,308.00,NaN,ips,yes,NaN,NaN,yes,14.5,yes,yes,223.00,1
548,intel uhd graphics,https://cellphones.com.vn/surface-laptop-go-21...,NaN,NaN,21.0,1.499000e+07,0.0,0.0,surface laptop go 12.4,microsoft,intel,1035g1,4.0,64.0,12.4,60.0,NaN,windows 10,21k-00019,1536 x 1024,no,NaN,NaN,1.11,NaN,no,278.18,NaN,ips,no,NaN,NaN,yes,15.69,yes,yes,205.67,1
549,intel uhd graphics,https://cellphones.com.vn/surface-laptop-go-25...,NaN,NaN,76.0,2.159000e+07,0.0,0.0,surface laptop go,microsoft,intel,1035g1,8.0,256.0,12.4,60.0,NaN,windows 10,NaN,1536 x 1024,no,NaN,aluminum,1.11,NaN,no,278.18,NaN,ips,no,NaN,ssd,yes,15.69,yes,yes,205.67,1
550,intel uhd graphics,https://cellphones.com.vn/surface-laptop-pro-7...,NaN,NaN,49.0,1.599000e+07,5.0,2.0,surface pro 7,microsoft,intel,1005g1,4.0,128.0,12.3,60.0,43.0,NaN,NaN,2736 x 1824,no,NaN,aluminum,0.00,NaN,no,292.00,NaN,NaN,no,NaN,ssd,yes,85,no,no,201.00,1


## Join merged result with LaptopArena (Failed)
Differences when merging with LaptopArena compared to previous part
- Condition of `Series` will be different
- df1 and df2 has different set of columns
- Mapping column may contains `NaN`


In [ ]:
text_facet(df_5,'Series')

---------- Series ------------
imac 24 2021 m1 i                                        5
macbook pro m1 2020                                      4
macbook air m1 2020                                      3
imac 27 2020                                             3
gaming g5 5510                                           2
vivobook flip tp412fa                                    2
ideapad 3 14alc6                                         2
surface laptop 3                                         2
modern 14 b11sb                                          2
macbook pro 13 touch bar 2020 i                          2
inspiron n3502 nk                                        2
modern 14 b11mo                                          2
ideapad 3 15itl6                                         2
imac 21.5 2020                                           2
ideapad 3 15alc6                                         2
15s fq2559tu 46m27pa                                     2
macbook pro 2021 m1 pro  

In [ ]:
print(len(df_6))
df_6.to_csv('merged_data.csv')

552


In [ ]:
df_a = merge(df_5,cleaned_laptoparena,pref1='_vnstore',pref2='_laptoparena',
             laptoparena_position='left',min_score=0.55,algo='custom')

KeyboardInterrupt: ignored

In [ ]:
similar_str('pavilion 15-eg0004tx 2d9b7pa',
            cleaned_laptoparena.Series["https://www.laptoparena.net/product/21721"],
            algo='custom')

In [ ]:
print(len(df_a))
print_merged_result(df_a,col='Series',prefs=('_vnstore','_laptoparena'))

In [ ]:
text_facet(df_a,'Release_date')